In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os
import torchvision.models as models
from torchvision import transforms
import pandas as pd

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
BATCH_SIZE=10

In [5]:
csv = pd.read_csv('./images/dataset/0_index.csv',usecols=[0,2,4,8])#图片路径、经纬长短纤、经纬氨纶、长焦图片路径（如果有）
csv_numpy = csv.to_numpy()

In [6]:
csv_numpy

array([['WIN_20200630_15_18_26_Pro.jpg', 's/s', '0/0', nan],
       ['WIN_20200630_15_18_47_Pro.jpg', 's/s', '0/0', nan],
       ['WIN_20200630_15_19_04_Pro.jpg', 's/s', '0/0', nan],
       ...,
       ['WIN_20200720_14_26_30_Pro.jpg', 'D/s', '0/0', nan],
       ['WIN_20200720_14_26_38_Pro.jpg', 'D/s', '0/0', nan],
       ['WIN_20200720_14_26_46_Pro.jpg', 'D/s', '0/0', nan]], dtype=object)

In [7]:
#四个二级if用于将对应图片的长焦图片放入数据列表中
#这里，如果不存在长焦路径，则为nan，nan可以理解为numpy、pandas中的None，当然在python看来和None不是一个东西，判断nan用np.isnan()
#但np.isnan()好像不能接受一个字符串参数？会报错，而nan在python中被当做是float。
#所以直接用isinstance。
ss,DD,sD,Ds = [],[],[],[]
for i in range(len(csv_numpy)):
    if csv_numpy[i][1] == 's/s':
        ss.append(csv_numpy[i][0])
        #if not isinstance(csv_numpy[i][3],float):
        #    ss.append(csv_numpy[i][3])
    elif csv_numpy[i][1] == 'D/D':
        DD.append(csv_numpy[i][0])
        #if not isinstance(csv_numpy[i][3],float):
        #    DD.append(csv_numpy[i][3])
    elif csv_numpy[i][1] == 'D/s':
        Ds.append(csv_numpy[i][0])
        #if not isinstance(csv_numpy[i][3],float):
        #    Ds.append(csv_numpy[i][3])
    elif csv_numpy[i][1] == 's/D':
        sD.append(csv_numpy[i][0])
        #if not isinstance(csv_numpy[i][3],float):
        #    sD.append(csv_numpy[i][3])
print(len(ss))
print(len(DD))
print(len(Ds))
print(len(sD))

225
188
41
31


In [8]:
train_ratio = 0.7
test_ratio = 0.3
train_set, test_set = [],[]
#这四个for以后想想能不能优化一下
for i in range(len(ss)):
    temp = ['./images/dataset/' + ss[i], 0]
    if i<int(train_ratio*len(ss)):
        train_set.append(temp)
    else:
        test_set.append(temp)
for i in range(len(DD)):
    temp = ['./images/dataset/' + DD[i], 1]
    if i<int(train_ratio*len(DD)):
        train_set.append(temp)
    else:
        test_set.append(temp)
for i in range(len(Ds)):
    temp = ['./images/dataset/' + Ds[i], 2]
    if i<int(train_ratio*len(Ds)):
        train_set.append(temp)
    else:
        test_set.append(temp)
for i in range(len(sD)):
    temp = ['./images/dataset/' + sD[i], 3]
    if i<int(train_ratio*len(sD)):
        train_set.append(temp)
    else:
        test_set.append(temp)
print(len(train_set),len(test_set))

337 148


In [9]:
class MyDataset(Dataset):
    def __init__(self, imgs, transform = None, target_transform = None):
        '''fh = open(txt_path, 'r')
        imgs = [] #内部为多个tuple，每个tuple为(图片路径，label)
        for line in fh:
            line = line.rstrip()
            words = line.split()
            imgs.append((words[0], int(words[1])))'''
        
        self.imgs = imgs * 3 #将数据暴力复制3份，后面会做transforms
        self.transform = transform
        self.target_transform = target_transform
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB') 
        if self.transform is not None:
            img = self.transform(img) 
        return img, label
    def __len__(self):
        return len(self.imgs)

In [10]:
#图像的初始化操作
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

#数据集加载方式设置
train_data = MyDataset(train_set, transform=train_transforms)
test_data = MyDataset(test_set, transform=test_transforms)

#由于样本不均衡，因此做重采样
from torch.utils.data.sampler import  WeightedRandomSampler
weights = []
for data,target in train_set:
    if target in (2,3):
        weights.append(4) #长短和短长的权重翻n倍
    else:
        weights.append(1)
sampler = WeightedRandomSampler(weights, num_samples=1700, replacement=True)#这里随便取了一个略小于总图片数量的batchsize的倍数

#然后就是调用DataLoader和刚刚创建的数据集，来创建dataloader，这里提一句，loader的长度是有多少个batch，所以和batch_size有关
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,num_workers=0,sampler=sampler)
#train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False,num_workers=0)
print('num_of_trainloader:', len(train_loader))
print('num_of_testloader:', len(test_loader))

num_of_trainloader: 170
num_of_testloader: 45


In [25]:
t = 0
d = 0
for (data,target) in train_loader:
    t=target
    d=data
    break
d.shape

torch.Size([10, 3, 224, 224])

In [11]:
def train(model,device, train_loader, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        x,y= data
        x=x.to(device)
        y=y.to(device)
        optimizer.zero_grad()
        y_hat= model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print ('Train Epoch: {}\t Loss: {:.6f}'.format(epoch,loss.item()))
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i,data in enumerate(test_loader):          
            x,y= data
            x=x.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            test_loss += criterion(y_hat, y).item() # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_data),
        100. * correct / len(test_data)))

In [12]:
#resnet18 = models.resnet18(pretrained=True)
#alexnet = models.alexnet(pretrained=True)
#vgg16 = models.vgg16(pretrained=True)

model_conv = models.alexnet(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.classifier[6].in_features
model_conv.classifier[6] = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params':model_conv.classifier[6].parameters()}
], lr=0.001)

In [13]:
for epoch in range(1, 10):
    train(model=model_conv, device=DEVICE, train_loader=train_loader,epoch=epoch)
    test(model=model_conv, device=DEVICE, test_loader=test_loader)

Train Epoch: 1	 Loss: 0.323903

Test set: Average loss: 0.0535, Accuracy: 51/69 (74%)

Train Epoch: 2	 Loss: 0.146808

Test set: Average loss: 0.0454, Accuracy: 48/69 (70%)

Train Epoch: 3	 Loss: 0.356405

Test set: Average loss: 0.0548, Accuracy: 54/69 (78%)

Train Epoch: 4	 Loss: 0.483915

Test set: Average loss: 0.0608, Accuracy: 54/69 (78%)

Train Epoch: 5	 Loss: 0.314868

Test set: Average loss: 0.0416, Accuracy: 57/69 (83%)

Train Epoch: 6	 Loss: 0.333603

Test set: Average loss: 0.0702, Accuracy: 51/69 (74%)

Train Epoch: 7	 Loss: 0.089547

Test set: Average loss: 0.0471, Accuracy: 54/69 (78%)

Train Epoch: 8	 Loss: 0.216139

Test set: Average loss: 0.0916, Accuracy: 51/69 (74%)

Train Epoch: 9	 Loss: 0.609754

Test set: Average loss: 0.0583, Accuracy: 54/69 (78%)



In [12]:
#resnet18 = models.resnet18(pretrained=True)
#alexnet = models.alexnet(pretrained=True)
#vgg16 = models.vgg16(pretrained=True)

model_conv = models.resnet18(pretrained=True)

param_num = 0
for i,param in enumerate(model_conv.parameters()):
    param_num += 1
for i,param in enumerate(model_conv.parameters()):
    if i >= (param_num-2):#解冻最后2层
        param.requires_grad = True
    else:
        param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 4)

model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam([
    {'params':model_conv.fc.parameters()}
], lr=0.001)

In [ ]:
for epoch in range(1, 20):
    train(model=model_conv, device=DEVICE, train_loader=train_loader,epoch=epoch)
    test(model=model_conv, device=DEVICE, test_loader=test_loader)

Train Epoch: 1	 Loss: 1.224071

Test set: Average loss: 0.0758, Accuracy: 498/690 (72%)

Train Epoch: 2	 Loss: 0.809651

Test set: Average loss: 0.0882, Accuracy: 417/690 (60%)

Train Epoch: 3	 Loss: 0.573209

Test set: Average loss: 0.0743, Accuracy: 489/690 (71%)

Train Epoch: 4	 Loss: 0.635795

Test set: Average loss: 0.0782, Accuracy: 465/690 (67%)



In [17]:
model_conv = models.resnet34(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params':model_conv.fc.parameters()}
], lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/yyh/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:03<00:00, 27.5MB/s]


In [18]:
for epoch in range(1, 10):
    train(model=model_conv, device=DEVICE, train_loader=train_loader,epoch=epoch)
    test(model=model_conv, device=DEVICE, test_loader=test_loader)

Train Epoch: 1	 Loss: 0.501376

Test set: Average loss: 0.0613, Accuracy: 42/69 (61%)

Train Epoch: 2	 Loss: 0.513787

Test set: Average loss: 0.0647, Accuracy: 48/69 (70%)

Train Epoch: 3	 Loss: 0.391931

Test set: Average loss: 0.0623, Accuracy: 48/69 (70%)

Train Epoch: 4	 Loss: 0.351778

Test set: Average loss: 0.0582, Accuracy: 42/69 (61%)

Train Epoch: 5	 Loss: 0.543261

Test set: Average loss: 0.0929, Accuracy: 42/69 (61%)

Train Epoch: 6	 Loss: 0.405109

Test set: Average loss: 0.0524, Accuracy: 45/69 (65%)

Train Epoch: 7	 Loss: 0.558961

Test set: Average loss: 0.0527, Accuracy: 48/69 (70%)

Train Epoch: 8	 Loss: 0.287587

Test set: Average loss: 0.0511, Accuracy: 51/69 (74%)

Train Epoch: 9	 Loss: 0.311003

Test set: Average loss: 0.0507, Accuracy: 51/69 (74%)



In [10]:
#resnet18 = models.resnet18(pretrained=True)
#alexnet = models.alexnet(pretrained=True)
#vgg16 = models.vgg16(pretrained=True)

model_conv = models.vgg16(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
#model_conv.classifier[0] = nn.Linear(25088, 4096)
num_second_last = model_conv.classifier[3].in_features
model_conv.classifier[3] = nn.Linear(num_second_last, num_second_last)
num_last = model_conv.classifier[6].in_features
model_conv.classifier[6] = nn.Linear(num_last, 2)

model_conv = model_conv.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam([
    {'params':model_conv.classifier[3].parameters()},
    {'params':model_conv.classifier[6].parameters()}
], lr=0.001)

In [17]:
model_conv

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
for epoch in range(1, 10):
    train(model=model_conv, device=DEVICE, train_loader=train_loader,epoch=epoch)
    test(model=model_conv, device=DEVICE, test_loader=test_loader)

Train Epoch: 1	 Loss: 0.039234

Test set: Average loss: 0.2682, Accuracy: 48/69 (70%)

Train Epoch: 2	 Loss: 0.014214

Test set: Average loss: 0.2211, Accuracy: 45/69 (65%)

Train Epoch: 3	 Loss: 0.005668

Test set: Average loss: 0.2541, Accuracy: 48/69 (70%)

Train Epoch: 4	 Loss: 0.008691

Test set: Average loss: 0.2853, Accuracy: 42/69 (61%)

Train Epoch: 5	 Loss: 0.132321

Test set: Average loss: 0.3135, Accuracy: 48/69 (70%)

Train Epoch: 6	 Loss: 2.570711

Test set: Average loss: 0.3454, Accuracy: 51/69 (74%)

Train Epoch: 7	 Loss: 3.714458

Test set: Average loss: 0.3400, Accuracy: 45/69 (65%)

Train Epoch: 8	 Loss: 1.539180

Test set: Average loss: 0.4139, Accuracy: 45/69 (65%)

Train Epoch: 9	 Loss: 0.571291

Test set: Average loss: 0.2755, Accuracy: 42/69 (61%)

